In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from Word2VecTransformer import Embedding_Word2Vec
from FastTextTransformer import FastTextTransformer
from sklearn.ensemble import RandomForestClassifier
from threading import Thread
from ClearTransformData import Cleardataset
import pandas as pd
import time
import pickle
import os

In [ ]:
model8 = [("fastText",FastTextTransformer(inputFile=Input,ft_home=ft_home,size=300)) ,("clfSVM",LinearSVC())]
param8={"fastText__model":["cbow","skipgram"],"clfSVM__C": [0.1, 1, 10, 100, 1000]}
model11 = [("fastText",FastTextTransformer(inputFile=Input,ft_home=ft_home,size=300)),("clfrandomForest",RandomForestClassifier())]
param11={"fastText__model":["cbow","skipgram"],"clfrandomForest__n_estimators":[200,500,700]}
